Importing needed libraries

In [2]:
!pip install pydantic

  Using cached pydantic-1.10.4-cp310-cp310-win_amd64.whl (2.1 MB)
  Using cached typing_extensions-4.4.0-py3-none-any.whl (26 kB)


You should consider upgrading via the 'C:\Users\akvil\.pyenv\pyenv-win\versions\3.10.0\python.exe -m pip install --upgrade pip' command.


In [23]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
import time
import random
from bs4 import BeautifulSoup 
import pandas as pd
import numpy as np
import re
from random import randrange
from tqdm import tqdm #progress bar
from typing import List, Dict, Optional
from pydantic import BaseModel

Create function to extract url, Brand and Model

In [44]:
results_url = []
results_base = []
results_p = []

In [62]:
def extract_base_info(url_list:str) -> Optional[List[Dict]]:

    chrome_options = webdriver.ChromeOptions()
    prefs = {"profile.managed_default_content_settings.images": 2}
    chrome_options.add_experimental_option("prefs", prefs)
    driver = webdriver.Chrome(ChromeDriverManager().install(), chrome_options=chrome_options)
    resp = driver.get(url_list)
    s = random.randint(2, 11)
    time.sleep(s)
    base_source = driver.page_source
    base_soup = BeautifulSoup(base_source, 'html.parser')
    cars_should_be = int((base_soup.find("span", {"class":"result-count"}).text.strip())[1:-1])
    count_cars = []
    url_list_class = base_soup.find_all("div", class_="auto-lists en")
    for j in url_list_class:
        
        link = j.find_all('a', href=True)
        for i in link:
            url=[i['href']]
            results_url.append(url)
        
        description = j.find_all("div", class_="announcement-title")
        for i in description:
            Description = i.text.strip()
            Make_model = Description.split(',')[0]
            Brand = Make_model.split(" ",1)[0]
            Model = Make_model.split(" ",1)[1]


            results_base.append({'Brand':Brand, 'Model':Model})
        price_list = j.find_all("div", class_="announcement-pricing-info")
        for k in price_list:
            pricee = k.text.strip()
            price = pricee.split('€')[0]
            results_p.append(price)
    return results_url, results_base, results_p
        

In [53]:
chrome_options = webdriver.ChromeOptions()
prefs = {"profile.managed_default_content_settings.images": 2}
chrome_options.add_experimental_option("prefs", prefs)
driver = webdriver.Chrome(ChromeDriverManager().install(), chrome_options=chrome_options)
resp = driver.get('https://en.autoplius.lt/ads/used-cars?page_nr=1')
s = random.randint(2, 11)
time.sleep(s)
base_source = driver.page_source
base_soup = BeautifulSoup(base_source, 'html.parser')
cars_should_be = int((base_soup.find("span", {"class":"result-count"}).text.strip())[1:-1])
count_cars = []
url_list_class = base_soup.find_all("div", class_="auto-lists en")
for j in url_list_class:
    
    link = j.find_all('a', href=True)
    for i in link:
        url=[i['href']]
        results_url.append(url)
    
    description = j.find_all("div", class_="announcement-title")
    for i in description:
        Description = i.text.strip()
        Make_model = Description.split(',')[0]
        Brand = Make_model.split(" ",1)[0]
        Model = Make_model.split(" ",1)[1]


        results_base.append({'Brand':Brand, 'Model':Model})
    price_list = j.find_all("div", class_="announcement-pricing-info")
    for k in price_list:
        pricee = k.text.strip()
        price = pricee.split('€')[0]
        results_p.append(price)

C:\Users\akvil\AppData\Local\Temp\ipykernel_22156\2282984223.py:4: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install(), chrome_options=chrome_options)
C:\Users\akvil\AppData\Local\Temp\ipykernel_22156\2282984223.py:4: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(ChromeDriverManager().install(), chrome_options=chrome_options)


Extract data using created function

In [69]:
results_url = []
results_base = []
results_p = []
for page in range(1,2):
    url_list = f"https://en.autoplius.lt/ads/used-cars?page_nr={page}"
    all_results = extract_base_info(url_list)
    results_url.extend(all_results[0])
    results_base.extend(all_results[1])
    results_p.extend(all_results[2])
    


C:\Users\akvil\AppData\Local\Temp\ipykernel_22156\470242013.py:6: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install(), chrome_options=chrome_options)
C:\Users\akvil\AppData\Local\Temp\ipykernel_22156\470242013.py:6: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(ChromeDriverManager().install(), chrome_options=chrome_options)


Generate Pandas data frame

In [71]:
df1 = pd.DataFrame(results_url)
df1.columns = ['Url']
df2 = pd.DataFrame.from_dict(results_base)
df3 = pd.DataFrame(results_p)
df3.columns = ['Price']
results = (df1.join(df2)).join(df3)
results.to_csv("test1.csv")

Extract data from each url

Generate extraction function

In [73]:
def extract_car_details(link:str) -> Optional[List[Dict]]:

    chrome_options = webdriver.ChromeOptions()
    prefs = {"profile.managed_default_content_settings.images": 2}
    chrome_options.add_experimental_option("prefs", prefs)

    driver = webdriver.Chrome(ChromeDriverManager().install(), chrome_options=chrome_options)
    t = random.randint(3, 6)
    driver.get(link)
    time.sleep(t)
    base_source1 = driver.page_source
    base_soup1 = BeautifulSoup(base_source1, 'html.parser')
    u = random.randint(2, 4)
    time.sleep(u)
    car_info = base_soup1.find_all("div", class_="parameter-row")
    labels = []
    values = []

    for group in base_soup1.find_all("div", class_='parameter-row'):
        labels.append([label.text.strip() for label in group.find_all("div", class_='parameter-label')])
        values.append([value.text.strip() for value in group.find_all("div", class_='parameter-value')])

    labels = [''.join(ele) for ele in labels][1:]
    values = [''.join(ele) for ele in values][1:]
    car_detail = {labels[k]: values[k] for k in range(len(labels))}

    return car_detail
    

extraxt data using function above

In [85]:
links = results['Url']
links

0     https://en.autoplius.lt/ads/audi-a8-4-2-l-salo...
1     https://en.autoplius.lt/ads/tesla-model-3-68-k...
2     https://en.autoplius.lt/ads/volkswagen-passat-...
3     https://en.autoplius.lt/ads/toyota-highlander-...
4     https://en.autoplius.lt/ads/kia-sportage-1-7-l...
5     https://en.autoplius.lt/ads/mercedes-benz-s550...
6     https://en.autoplius.lt/ads/renault-kangoo-1-5...
7     https://en.autoplius.lt/ads/bmw-116-hatchback-...
8     https://en.autoplius.lt/ads/bmw-318-gran-turis...
9     https://en.autoplius.lt/ads/toyota-hilux-3-0-l...
10    https://en.autoplius.lt/ads/bmw-535-3-0-l-salo...
11    https://en.autoplius.lt/ads/skoda-kodiaq-2-0-l...
12    https://en.autoplius.lt/ads/bmw-320-2-0-l-wago...
13    https://en.autoplius.lt/ads/nissan-qashqai-1-5...
14    https://en.autoplius.lt/ads/bmw-120-2-0-l-hatc...
15    https://en.autoplius.lt/ads/bmw-525-3-0-l-wago...
16    https://en.autoplius.lt/ads/bmw-116-2-0-l-hatc...
17    https://en.autoplius.lt/ads/mini-cooper-1-

In [86]:
cars_details = []
for link in links:
    cars_details.append(extract_car_details(link))

C:\Users\akvil\AppData\Local\Temp\ipykernel_22156\782093128.py:7: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install(), chrome_options=chrome_options)
C:\Users\akvil\AppData\Local\Temp\ipykernel_22156\782093128.py:7: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(ChromeDriverManager().install(), chrome_options=chrome_options)


In [87]:
cars_details


[{'Date of manufacture': '2011-04',
  'Mileage': '340 000 km',
  'Engine': '4163 cm³, 373 HP (274kW)',
  'Fuel type': 'Petrol',
  'Body type': 'Saloon / sedan',
  'Number of doors': '4/5',
  'Driven wheels': 'All wheel (4х4)',
  'Gearbox': 'Automatic',
  'Color': 'Gray / silver',
  'Damage': 'Crashed',
  'Steering wheel': 'Left hand drive (LHD)',
  'Kerb weight, kg': '1835',
  'Number of seats': '5',
  'VIN number': 'WAUZZZ4H...\n            Show',
  'Owner declaration code': 'SDK code is optional (car not in Lithuania)',
  'Euro standard': 'Euro 5',
  'CO₂ emisija, g/km': '219 g/km',
  'Pollution tax in Lithuania': '182.11 €',
  'Urban': '13,3',
  'Extra-urban': '7,2',
  'Combined': '9,5'},
 {'Date of manufacture': '2019-04',
  'Mileage': '57 438 km',
  'Fuel type': 'Electricity',
  '': 'Rechargeable',
  'Body type': 'Hatchback',
  'Number of doors': '4/5',
  'Driven wheels': 'Rear wheel drive (RWD)',
  'Gearbox': 'Automatic',
  'Color': 'White',
  'Damage': 'Crashed',
  'Steering whe

In [89]:
add_pd = pd.DataFrame.from_dict(cars_details)
add_pd

,Date of manufacture,Mileage,Engine,Fuel type,Body type,Number of doors,Driven wheels,Gearbox,Color,Damage,...,Extra-urban,Combined,,"Battery capacity, kWh",Electric range,Export,Climate control,MOT test expiry,Wheel size,First registration country
0,2011-04,340 000 km,"4163 cm³, 373 HP (274kW)",Petrol,Saloon / sedan,4/5,All wheel (4х4),Automatic,Gray / silver,Crashed,...,"7,2","9,5",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2019-04,57 438 km,NaN,Electricity,Hatchback,4/5,Rear wheel drive (RWD),Automatic,White,Crashed,...,NaN,NaN,Rechargeable,68 kWh,409 km,NaN,NaN,NaN,NaN,NaN
2,2011,287 100 km,"1968 cm³, 140 HP (103kW)",Diesel,Wagon,4/5,Front wheel drive (FWD),Automatic,Blue,NaN,...,"4,5","5,2",NaN,NaN,NaN,6 820 €,Air conditioning,2023-09,R17,Germany
3,2015,174 000 km,"3456 cm³, 273 HP (201kW)",Petrol / LPG,SUV / off-road,4/5,NaN,Automatic,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024-08,NaN,NaN
4,2016,175 000 km,"1700 cm³, 116 HP (85kW)",Diesel,SUV / off-road,4/5,NaN,Manual,White,Engine damage,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023-03,NaN,NaN
5,2014-11,231 189 km,"4664 cm³, 449 HP (330kW)",Petrol,Saloon / sedan,4/5,All wheel (4х4),Automatic,Black,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Climate control,2023-06,R20,NaN
6,2013-07,211 267 km,"1461 cm³, 90 HP (66kW)",Diesel,MPV / minivan,4/5,Front wheel drive (FWD),Manual,White,NaN,...,NaN,NaN,NaN,NaN,NaN,4 900 €,Air conditioning,2023-10,R15,Lithuania
7,2012-05,184 892 km,116 HP (85kW),Diesel,Hatchback,4/5,NaN,Automatic,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024,NaN,NaN
8,2016,155 800 km,"1995 cm³, 150 HP (110kW)",Diesel,Hatchback,4/5,Rear wheel drive (RWD),Automatic,Brown / beige,NaN,...,4,5,NaN,NaN,NaN,NaN,Climate control,2023-07,R17,Germany
9,2015,110 000 km,"3000 cm³, 171 HP (126kW)",Diesel,Pick-up,4/5,All wheel (4х4),Automatic,Yellow / gold,NaN,...,7,8,NaN,NaN,NaN,20 250 €,Climate control,2023-11,R17,Lithuania


In [90]:
all_results = results.join(add_pd)
all_results.to_csv('cars.csv')